In [135]:
import sys
from ipychart import chart
import pandas as pd

> Example of Chart.js graphs : https://tobiasahlin.com/blog/chartjs-charts-to-get-you-started/

# I. Work with python objects (dict, list)

## One dataset charts

**1. Bar Chart**

In [136]:
dataset = {
    'labels': ['Germany','Spain', 'UK', 'Italy', 'Norway', 'France', 'Poland', 'Portugal', 'Sweden', 'Ireland'],
    'datasets': [{ 
            'data': [114,106,106,107,111,133, 109, 109, 108, 107],

}]}

options = {'title': {
            'display': True, 
            'text': 'Bar Chart', 
            'fontSize': 22}}

mychart = chart.Chart(dataset, 'bar', options=options)
mychart

Chart(layout=Layout(align_self='stretch', height='auto'))

**2. Line Chart**

In [137]:
dataset = {
    'labels': ['Germany','Spain', 'UK', 'Italy', 'Norway', 'France', 'Poland', 'Portugal', 'Sweden', 'Ireland'],
    'datasets': [{ 
            'data': [114,106,106,107,111,133, 109, 109, 108, 107],
            'backgroundColor': 'rgba(75, 192, 192, 0.2)',
}]}

options = {'title': {
            'display': True, 
            'text': 'Line Chart', 
            'fontSize': 22}}

mychart = chart.Chart(dataset, 'line', options=options)
mychart

Chart(layout=Layout(align_self='stretch', height='auto'))

**3. Radar Chart**

In [138]:
dataset = {
    'labels': ['Germany','Spain', 'UK', 'Italy', 'Norway', 'France', 'Poland', 'Portugal', 'Sweden', 'Ireland'],
    'datasets': [{ 
            'data': [114,106,106,107,111,133, 109, 109, 108, 107],
            'backgroundColor': 'rgba(153, 102, 255, 0.2)',

}]}

options = {'title': {
            'display': True, 
            'text': 'Radar Chart', 
            'fontSize': 22}}

mychart = chart.Chart(dataset, 'radar', options=options)
mychart

Chart(layout=Layout(align_self='stretch', height='auto'))

**4. Bubble Chart**

In [139]:
dataset= {
    'labels': ["Company 1", "Company 2", "Company 3", "Company 4"],
    'datasets': [{
                'data': [{'x': 5, 'y': 10, "r": 10}, {'x': 15, 'y': 3, 'r': 15}, {'x': 7, 'y': 15, 'r': 30}, {'x': 25, 'y': 5, 'r': 10}],    }]}
        

options = {'title': {
            'display': True, 
            'text': 'Bubble Chart', 
            'fontSize': 22}}

mychart = chart.Chart(dataset,  'bubble',  options=options)
mychart

Chart(layout=Layout(align_self='stretch', height='auto'))

**5. PolarArea, Doughtnut and Pie Charts**

In [143]:
dataset =  {
    'labels': ['Germany','Spain', 'UK', 'Italy', 'Norway', 'France', 'Poland', 'Portugal', 'Sweden', 'Ireland'],
    'datasets': [{ 
            'data': [114,106,106,107,111,133, 109, 109, 108, 107],
        
}]}

options = {'title': {
            'display': True, 
            'text': 'Polar Area Chart', 
            'fontSize': 22}}

mychart = chart.Chart(dataset, 'polarArea', options=options) # try also 'doughnut', 'pie'
mychart

Chart(layout=Layout(align_self='stretch', height='auto'))

## Multiple datasets charts

**1. Bar Chart**

In [153]:
dataset = {
    'labels': [1500, 1600, 1700, 1750, 1800, 1850, 1900, 1950, 1999, 2050], 
    'datasets': [
        { 'data': [86, 114, 106, 106, 107, 111, 133, 221, 783, 2478], 
          'label': "Africa", }, 
        { 'data': [282, 350, 411, 502, 635, 809, 947, 1402, 3700, 5267], 
          'label': "Asia"}, 
        {'data': [168, 170, 178, 190, 203, 276, 408, 547, 675, 734], 
         'label': "Europe"}, 
        {'data': [40, 20, 10, 16, 24, 38, 74, 167, 508, 784], 
         'label': "Latin America"}, 
        {'data': [6, 3, 2, 2, 7, 26, 82, 172, 312, 433], 
         'label': "North America"}
    ]}

options = {'title': {
            'display': True, 
            'text': 'Bar Chart with multiple datasets', 
            'fontSize': 22}}

mychart = chart.Chart(dataset,  'bar',  options=options)
mychart

Chart(layout=Layout(align_self='stretch', height='auto'))

**2. Line Chart**

In [144]:
dataset = {
    'labels': [1500,1600,1700,1750,1800,1850,1900,1950,1999,2050],
    'datasets': [{ 
        'data': [86,114,106,106,107,111,133,221,783,2478],
        'label': "Africa",
        'fill': False
      }, { 
        'data': [282,350,411,502,635,809,947,1402,3700,5267],
        'label': "Asia",
        'fill': False
      }, { 
        'data': [168,170,178,190,203,276,408,547,675,734],
        'label': "Europe",
        'fill': False
      }, { 
        'data': [40,20,10,16,24,38,74,167,508,784],
        'label': "Latin America",
        'fill': False
      }, { 
        'data': [6,3,2,2,7,26,82,172,312,433],
        'label': "North America",
        'fill': False
      }
    ]
  }
        
options = {'title': {'display': True, 'text': 'Line Chart with multiple datasets', 'fontSize': 22}, 
           'legend': {'position': 'bottom'}}

mychart = chart.Chart(dataset,  'line',  options=options)
mychart

Chart(layout=Layout(align_self='stretch', height='auto'))

**3. Radar Chart**

In [109]:
dataset = {
    'labels': [1500,1600,1700,1750,1800,1850,1900,1950,1999,2050],
    'datasets': [{ 
        'data': [86,114,106,106,107,111,133,221,121,142],
        'label': "Africa",
        'fill': False
      }, { 
        'data': [99,130,64,100,73,22,88,198,144,64],
        'label': "Asia",
        'fill': False
      }, { 
        'data': [168,170,178,190,203,200,164,100,72,85],
        'label': "Europe",
        'fill': False
      }, { 
        'data': [40,20,10,16,24,38,74,167,80,150],
        'label': "Latin America",
        'fill': False
      }, { 
        'data': [56,95,44,112,215,35,95,74,64,78],
        'label': "North America",
        'fill': False
      }
    ]
  }
        
options = {'title': {'display': True, 'text': 'Radar Chart with multiple datasets', 'fontSize': 22}, 
           'legend': {'position': 'right', 'align': 'center'}}

mychart = chart.Chart(dataset,  'radar',  options=options)
mychart

Chart(layout=Layout(align_self='stretch', height='auto'))

## Mixed types charts

In [158]:
dataset = {
    'labels': [1500,1600,1700,1750,1800,1850,1900,1950,1999,2050],
    'datasets': [{ 
        'data': [86,114,106,106,107,111,133,221,121,142],
        'label': "Africa",
        'fill': False,
        'type': 'line'
      }, { 
        'data': [99,130,64,100,73,22,88,198,144,64],
        'label': "Asia",
        'fill': False
      }, { 
        'data': [168,170,178,190,203,200,164,100,72,85],
        'label': "Europe",
        'type': 'line'
      }, { 
        'data': [40,20,10,16,24,38,74,167,80,150],
        'label': "Latin America",
        'fill': False
      }, { 
        'data': [56,95,44,112,215,35,95,74,64,78],
        'label': "North America",
        'fill': False,
        'type': 'horizontalBar'
      }
    ]
  }
        
options = {'title': {'display': True, 'text': 'Mixed types chart', 'fontSize': 22}, 
           'legend': {'position': 'right', 'align': 'center'}}

mychart = chart.Chart(dataset,  'bar',  options=options)
mychart

Chart(layout=Layout(align_self='stretch', height='auto'))

# II. Work with pandas dataframes

In [112]:
cars = {'Brand': ['Honda Civic', 'Toyota Corolla', 'Ford Focus', 'Audi A4'],
        'Price': [22000, 25000, 27000, 35000],
        'Year': [2015, 2013, 2018, 2018]
        }
 
df = pd.DataFrame(cars, columns= ['Brand', 'Price','Year'])

df

,Brand,Price,Year
0,Honda Civic,22000,2015
1,Toyota Corolla,25000,2013
2,Ford Focus,27000,2018
3,Audi A4,35000,2018


In [128]:
mychart = chart.Chart(data=df, kind='line', x='Brand', y='Price')
mychart

Chart(layout=Layout(align_self='stretch', height='auto'))